# Breast Cancer Prediction 

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)

---

## Background
For this illustration, we have taken an example for breast cancer prediction using UCI'S breast cancer diagnostic data set available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 

---

## Setup

Now we'll import the Python libraries we'll need.

In [9]:
!pip3 install tensorflow --user
!pip3 install autokeras --user

You are using pip version 19.0.2, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/91/32/86ad97a008d73eb438e0efd153075a802c7a5d89eb5d6da8ee0812d3c544/autokeras-1.0.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cd/80/5bb262050dd2f30f8819626b7c92339708fe2ed7bd5554c8193b4487b367/tqdm-4.42.1-py2.py3-none-any.whl
  The script tqdm is installed in '/home/ec2-user/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The script tabulate is installed in '/home/ec2-user/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You are using pip version 19.0.2, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' com

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json

import autokeras as ak

---
## Data

Data Source: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
        https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [11]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header = None)

# specify columns extracted from wbdc.names
data.columns = ["id","diagnosis","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean",
                "compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean",
                "radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se",
                "concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst",
                "perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst",
                "concave points_worst","symmetry_worst","fractal_dimension_worst"] 

# save the data
data.to_csv("data.csv", sep=',', index=False)

# print the shape of the data file
print(data.shape)

# show the top few rows
display(data.head())

# describe the data object
display(data.describe())

# we will also summarize the categorical field diganosis 
display(data.diagnosis.value_counts())


(569, 32)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


B    357
M    212
Name: diagnosis, dtype: int64

#### Key observations:
* Data has 569 observations and 32 columns.
* First field is 'id'.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features available for prediction.

## Create Features and Labels
#### Split the data into 80% training, 10% validation and 10% testing.

In [12]:
y = ((data.iloc[:,1] == 'M') +0).values;
X = data.iloc[:,2:].values;

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

---
## Train


In [13]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(max_trials=10) # It tries 10 different models.

clf.fit(X_train, y_train)


Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/graph.py", line 291, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/engine/block.py", line 40, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.py", line 157, in build
    output_node = self.build_body(hp, output_node)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.p

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/graph.py", line 291, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/engine/block.py", line 40, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.py", line 157, in build
    output_node = self.build_body(hp, output_node)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.p

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/graph.py", line 291, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/engine/block.py", line 40, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.py", line 157, in build
    output_node = self.build_body(hp, output_node)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.p

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/graph.py", line 291, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/engine/block.py", line 40, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.py", line 157, in build
    output_node = self.build_body(hp, output_node)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.p

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/graph.py", line 291, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/engine/block.py", line 40, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.py", line 157, in build
    output_node = self.build_body(hp, output_node)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.p

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/graph.py", line 291, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/engine/block.py", line 40, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.py", line 157, in build
    output_node = self.build_body(hp, output_node)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/autokeras/hypermodels/wrapper.p

RuntimeError: Too many failed attempts to build model.

In [14]:
print(clf.evaluate(X_val, y_val))

IndexError: list index out of range